In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

from pytorch_lightning import utilities as pl_utils
from pytorch_lightning.trainer.trainer import Trainer
from pytorch_lightning.plugins import DDPPlugin
import torch
import pathlib
from functools import partial
import sys, glob
import numpy as np

sys.path.append('..')
sys.path.append('../deep-learning-base')
sys.path.append('../deep-learning-base/training')
sys.path.append('../partially_inverted_reps')

import plot_helper as plt_hp
import output as out
from training import LitProgressBar, NicerModelCheckpointing
import training.finetuning as ft
import architectures as arch
from architectures.callbacks import LightningWrapper, LinearEvalWrapper
from attack.callbacks import AdvCallback
from datasets.data_modules import DATA_MODULES
import datasets.dataset_metadata as dsmd
from partial_loss import PartialInversionLoss, PartialInversionRegularizedLoss
from __init__ import DATA_PATH_IMAGENET, DATA_PATH, SERVER_PROJECT_PATH

/NS/twitter_archive2/work/vnanda/minconda3/envs/dl_base/lib/python3.7/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
CHECKPOINT_PATHS = {
    'resnet50_ff2048': '/NS/robustness_2/work/vnanda/invariances_in_reps/'
                       'deep-learning-base/checkpoints/imagenet1k/r50_mrl0_e0_ff2048.pt',
    'wide_resnet50_2': '/NS/robustness_3/work/vnanda/adv-robustness/logs/'\
                       'robust_imagenet/eps0/wideresnet-50-2-nonrob.ckpt',
    'wide_resnet50_4': '/NS/robustness_3/work/vnanda/adv-robustness/logs/'\
                       'robust_imagenet/eps0/wideresnet-50-4-nonrob.ckpt',
    'vgg16_bn': '',
    'resnet18': '',
    'resnet50': '',
    'vit_small_patch16_224': \
        '/NS/robustness_2/work/vnanda/invariances_in_reps/deep-learning-base/checkpoints/imagenet1k/'\
        'S_16-i1k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0.npz',
    'vit_small_patch32_224': \
        '/NS/robustness_2/work/vnanda/invariances_in_reps/deep-learning-base/checkpoints/imagenet1k/'\
        'S_32-i1k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0.npz'
}
SEED = 2
NUM_NODES = 1
DEVICES = 1
BATCH_SIZE = 200

In [3]:
def evaluate(model, checkpoint_path, dataset='imagenet'):
    dm = DATA_MODULES[dataset](
        data_dir=DATA_PATH_IMAGENET if 'imagenet' in dataset else DATA_PATH,
        transform_train=dsmd.TEST_TRANSFORMS_DEFAULT(224),
        transform_test=dsmd.TEST_TRANSFORMS_DEFAULT(224),
        batch_size=BATCH_SIZE)
    dm.init_remaining_attrs(dataset)
    
    m1 = arch.create_model(model, dataset, pretrained=True,
                           checkpoint_path=checkpoint_path, seed=SEED, 
                           num_classes=dsmd.DATASET_PARAMS[dataset]['num_classes'],
                           callback=partial(LightningWrapper, 
                                            dataset_name=dataset,
                                            inference_kwargs={'with_latent': True}),
                           loading_function_kwargs={'strict': True}) # some final layers are strange
    pl_utils.seed.seed_everything(SEED, workers=True)

    trainer = Trainer(accelerator='gpu', 
                      devices=DEVICES,
                      num_nodes=NUM_NODES,
                      log_every_n_steps=1,
                      auto_select_gpus=True, 
                      deterministic=True,
                      check_val_every_n_epoch=1,
                      num_sanity_val_steps=0,
                      callbacks=[
                        LitProgressBar(['loss', 
                                        'running_test_acc'])])

    out = trainer.predict(m1, dataloaders=[dm.test_dataloader()])
    return torch.sum(torch.argmax(out[0], 1) == out[2]) / len(out[2])

In [ ]:
for model, checkpoint_path in CHECKPOINT_PATHS.items():
    print (f'model: {model}, test acc: {evaluate(model.split("-")[0], checkpoint_path).item()}')

Global seed set to 2
No pretrained weights exist or were found for this model. Using random initialization.
Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


Predicting: 0it [00:00, ?it/s]

Global seed set to 2


model: resnet50_ff2048, test acc: 0.7652199864387512


Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


...

Predicting: 0it [00:00, ?it/s]

Global seed set to 2


model: resnet50, test acc: 0.8006799817085266


Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


Predicting: 0it [00:00, ?it/s]